In [1]:
# Perform_AI.src.data_processing.py

import streamlit as st
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import csv
import os
import sys
import plotly.graph_objs as go
import plotly.io as pio

In [2]:
import sys
from pathlib import Path

# Set the root directory as the path, assuming Perform_AI is the project root
sys.path.append(str(Path().resolve().parent))

In [3]:
from src.calorie_calculations import calculate_total_calories

from src.calorie_estimation_models import estimate_calories_with_duration, estimate_calories_with_nixtla
from src.tss_calculations import calculate_total_tss_and_metrics_from_tss

from src.data_loader.files_extracting import FileLoader
from src.data_loader.files_saving import FileSaver
from params import CLOUD_ON, GIVEN_DATE, BEST_MODEL

In [4]:
def clean_data_basic(df):
    """
    Clean data for the given dataframes.

    Parameters:
        dfs (dict): Dictionary of DataFrames to clean.
        date_cols (dict): Dictionary mapping DataFrame names to their date column names.
    """
    df = df.replace('--', np.nan)
    df = df.drop_duplicates()

    return df

In [5]:
def convert_to_datetime(df, date_col):
    """
    Convert specified column to datetime and set as index.

    Parameters:
        df (pd.DataFrame): DataFrame to process.
        date_col (str): Name of the date column.
    """
    if date_col != 'Timestamp':
        df['Date'] = pd.to_datetime(df[date_col])
        df = df.sort_values('Date')
        df = df.set_index('Date')
    else:
        df.index.name = 'Date'
        df = df.sort_values('Date')
        df = df.set_index(pd.to_datetime(df.index))

    if date_col in df.columns:
        df = df.drop(columns=date_col)
    return df

In [6]:
def clean_activities(df):
    """
    Clean activity data to keep relevant columns and rename them.

    Parameters:
        df (pd.DataFrame): DataFrame containing activity data.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    columns_to_keep = ["Type d'activité", 'Distance', 'Calories', 'Durée', 'Fréquence cardiaque moyenne']
    df = df[columns_to_keep].copy().rename(columns={
        'Distance': 'DistanceInMeters',
        'Durée': 'TimeTotalInHours',
        'Fréquence cardiaque moyenne': 'HeartRateAverage',
        'Type d\'activité': 'WorkoutType'
    })

    df['HeartRateAverage'] = pd.to_numeric(df['HeartRateAverage'], errors='coerce')
    df = df[df['HeartRateAverage'].notna()]

    df = df[~df["WorkoutType"].isin(['HIIT', 'Exercice de respiration', 'Musculation'])].copy()

    sports_types = {
        'Nat. piscine': 'Swim',
        'Cyclisme': 'Bike',
        'Course à pied': 'Run',
        "Vélo d'intérieur": 'Bike',
        'Cyclisme virtuel': 'Bike',
        'Course à pied sur tapis roulant': 'Run',
        'Natation': 'Swim',
    }
    df["WorkoutType"] = df["WorkoutType"].apply(lambda x: sports_types[x])

    # Convert Durée from 'hh:mm:ss' to total minutes
    df['TimeTotalInHours'] = pd.to_timedelta(df['TimeTotalInHours']).dt.total_seconds() / 3600  # Convert to Hours

    # Convert relevant columns to numeric (remove commas, etc.)
    df['DistanceInMeters'] = pd.to_numeric(df['DistanceInMeters'].str.replace(',', '.'), errors='coerce')
    df['Calories'] = pd.to_numeric(df['Calories'], errors='coerce')

    # Drop rows with NaN values in critical columns
    df = df.dropna(subset=['DistanceInMeters', 'Calories', 'TimeTotalInHours', 'HeartRateAverage'])

    # df = df[df['DistanceInMeters']>0].copy() # NOTE: not needed since, i will be using only TotalDuration or TimeTotalInHours

    return df

In [7]:

def filter_workouts_and_remove_nans(df, given_date = GIVEN_DATE):
    columns_to_keep_workouts = ['WorkoutType', 'Title', 'WorkoutDescription', 'CoachComments', 'HeartRateAverage', 'TimeTotalInHours', 'DistanceInMeters', 'PlannedDuration', 'PlannedDistanceInMeters']
    df = df[columns_to_keep_workouts].copy()

    before_df = df[df.index < given_date].copy()
    after_df = df[df.index >= given_date].copy()
    # Remove rows, before the given date, where i didn't train, meaning, where HR and Total Time is nan.
    before_df_cleaned = before_df[~(before_df['HeartRateAverage'].isna() & before_df['TimeTotalInHours'].isna())].copy() # NOTE: HERE IS THE PART THAT CAUSES THE WEIRD BEHAVIOUR. Explanation below
    # TODO: (BTW, I DON'T NEED TO REMOVE THE HEARTRATEAVERAGE.ISNA, since what's important for me is timetotalinhours only)

    # Remove rows, after the given date, where Planned Duration is nan, which means there is no info on training, so no tss
    after_df = after_df[after_df['PlannedDuration'].notna()]

    # Concatenate before and after dataframes
    w_df = pd.concat([before_df_cleaned, after_df])
    # Keep dates where there was a Run Swim or Bike training Plan
    w_df = w_df[w_df['WorkoutType'].isin(['Run', 'Swim', 'Bike'])]

    # Fill NaN values in object columns with an empty string
    object_cols = w_df.select_dtypes(include=['object']).columns
    w_df[object_cols] = w_df[object_cols].fillna('')

    return w_df

In [8]:
def standardize_date_index(df):
    """
    Converts the index of the dataframe to datetime and formats it as 'YYYY-MM-DD'.

    Parameters:
    df (pd.DataFrame): DataFrame with a date index to be standardized.

    Returns:
    pd.DataFrame: DataFrame with the index formatted as 'YYYY-MM-DD'.
    """
    # Convert index to datetime
    df.index = pd.to_datetime(df.index)
    # Format index as 'YYYY-MM-DD'
    df.index = df.index.strftime('%Y-%m-%d')
    return df


In [9]:
dataframes_names = {
    'workouts': ['tp_workouts_2022-03-03_to_2023-03-03', 'tp_workouts_2023-03-03_to_2024-03-03', 'tp_workouts_2024-03-03_to_2025-03-03'],
    'activities': 'activities',
    'foods': [f"FOOD-DATA-GROUP{i}" for i in range(1,6)]
}

workouts_df = pd.concat([FileLoader()._load_csv('data/raw/csv', name) for name in dataframes_names['workouts']], ignore_index=True)
activities_df = FileLoader()._load_csv('data/raw/csv', dataframes_names['activities'])
foods = pd.concat([FileLoader()._load_csv('data/raw/csv', name, index=0) for name in dataframes_names['foods']], ignore_index=True)

2024-11-01 10:55:16,671 - INFO - Tp Workouts 2022-03-03 To 2023-03-03 dataframe loaded successfully from /Users/juanpabloangaritaafricano/code/juanpabloangarita/Perform_AI/data/raw/csv/tp_workouts_2022-03-03_to_2023-03-03.csv
2024-11-01 10:55:16,674 - INFO - Tp Workouts 2023-03-03 To 2024-03-03 dataframe loaded successfully from /Users/juanpabloangaritaafricano/code/juanpabloangarita/Perform_AI/data/raw/csv/tp_workouts_2023-03-03_to_2024-03-03.csv
2024-11-01 10:55:16,678 - INFO - Tp Workouts 2024-03-03 To 2025-03-03 dataframe loaded successfully from /Users/juanpabloangaritaafricano/code/juanpabloangarita/Perform_AI/data/raw/csv/tp_workouts_2024-03-03_to_2025-03-03.csv
2024-11-01 10:55:16,686 - INFO - Activities dataframe loaded successfully from /Users/juanpabloangaritaafricano/code/juanpabloangarita/Perform_AI/data/raw/csv/activities.csv
2024-11-01 10:55:16,691 - INFO - Food-Data-Group1 dataframe loaded successfully from /Users/juanpabloangaritaafricano/code/juanpabloangarita/Perform

In [10]:
dataframes = {
    'Date': activities_df,
    #'Date': sleep_df,
    #'Timestamp': health_metrics_df, # already as index
    'WorkoutDay': workouts_df
}

In [11]:
workouts_df = clean_data_basic(workouts_df).copy()

In [12]:
workouts_df = convert_to_datetime(workouts_df, 'WorkoutDay').copy()

In [13]:
activities_df = clean_data_basic(activities_df).copy()

In [14]:
activities_df = convert_to_datetime(activities_df, 'Date').copy()

In [14]:
def filter_and_translate_columns(df, column_mapping, columns_to_keep):
    """
    Translates column names in a DataFrame based on a given mapping and filters to keep only specified columns.

    Parameters:
    - df (pd.DataFrame): The DataFrame with original column names.
    - column_mapping (dict): A dictionary mapping original column names to desired column names.
    - columns_to_keep (list, optional): List of columns to keep in the final DataFrame after renaming. Defaults to None.

    Returns:
    - pd.DataFrame: A DataFrame with renamed and filtered columns.
    """
    # Translate columns
    df_translated = df.rename(columns=column_mapping)

    # Determine columns to keep
    df_translated = df_translated[columns_to_keep]

    return df_translated


In [15]:
columns_to_keep_workouts = ['WorkoutType', 'Title', 'WorkoutDescription', 'CoachComments', 
                            'HeartRateAverage', 'TimeTotalInHours', 'DistanceInMeters', 'PlannedDuration', 'PlannedDistanceInMeters']

In [16]:
french_to_english = {
    'Type d\'activité': 'WorkoutType',
    'Titre': 'Title',
    'Fréquence cardiaque moyenne': 'HeartRateAverage',
    'Durée': 'TimeTotalInHours',
    'Distance': 'DistanceInMeters',
    'Calories': 'Calories'
}

In [17]:
columns_to_keep_activities = list(french_to_english.values())

In [18]:
workouts_df = filter_and_translate_columns(workouts_df, {}, columns_to_keep_workouts).copy()

In [19]:
activities_df = filter_and_translate_columns(activities_df, french_to_english, columns_to_keep_activities).copy()

In [20]:
workouts_df['WorkoutType'].unique()

array(['Other', 'Run', 'Strength', 'Swim', 'Bike', 'Brick', 'Day Off'],
      dtype=object)

In [21]:
activities_df['WorkoutType'].unique()

array(['Cyclisme', 'Course à pied', 'Nat. piscine', 'Cyclisme virtuel',
       "Vélo d'intérieur", 'Musculation',
       'Course à pied sur tapis roulant', 'HIIT', 'Natation',
       'Exercice de respiration'], dtype=object)

In [30]:
def filter_and_translate_workouts(df, workouts_to_remove, sports_mapping=None):
    """
    Filters and translates workout types in a DataFrame based on specified criteria.

    Parameters:
    - df (pd.DataFrame): The DataFrame with workout data.
    - workouts_to_remove (list): List of workout types to exclude from the DataFrame.
    - sports_mapping (dict, optional): A dictionary to translate workout types. Defaults to None.

    Returns:
    - pd.DataFrame: The DataFrame with filtered and translated workout types.
    """
    # Filter out unwanted workout types
    df_filtered = df[~df['WorkoutType'].isin(workouts_to_remove)].copy()

    # Apply mapping if provided
    if sports_mapping:
        df_filtered['WorkoutType'] = df_filtered['WorkoutType'].map(sports_mapping).fillna(df_filtered['WorkoutType'])

    return df_filtered


In [31]:
workouts_to_remove_both_dfs = ['Other', 'Strength', 'Day Off', 'HIIT', 'Exercice de respiration', 'Musculation']

In [32]:
sports_types = {
    'Nat. piscine': 'Swim',
    'Cyclisme': 'Bike',
    'Course à pied': 'Run',
    "Vélo d'intérieur": 'Bike',
    'Cyclisme virtuel': 'Bike',
    'Course à pied sur tapis roulant': 'Run',
    'Natation': 'Swim',
}

In [33]:
workouts_df = filter_and_translate_workouts(workouts_df, workouts_to_remove_both_dfs).copy()

In [34]:
activities_df = filter_and_translate_workouts(activities_df, workouts_to_remove_both_dfs, sports_types).copy()

In [36]:
workouts_df['WorkoutType'].unique()

array(['Run', 'Swim', 'Bike', 'Brick'], dtype=object)

In [37]:
activities_df['WorkoutType'].unique()

array(['Bike', 'Run', 'Swim'], dtype=object)

In [38]:
workouts_df.head()

,WorkoutType,Title,WorkoutDescription,CoachComments,HeartRateAverage,TimeTotalInHours,DistanceInMeters,PlannedDuration,PlannedDistanceInMeters
Date,,,,,,,,,
2022-03-13,Run,Running,NaN,NaN,147.0,0.227822,2190.780029,NaN,NaN
2022-03-14,Run,Running,NaN,NaN,126.0,0.317827,2363.750000,NaN,NaN
2022-03-15,Run,Running,NaN,NaN,162.0,0.366062,3018.330078,NaN,NaN
2022-03-16,Run,Running,NaN,NaN,151.0,0.500626,3558.080078,NaN,NaN
2022-03-19,Run,Running,NaN,NaN,138.0,0.093556,760.710022,NaN,NaN


In [39]:
activities_df.head()

,WorkoutType,Title,HeartRateAverage,TimeTotalInHours,DistanceInMeters,Calories
0,Bike,Vincennes - Base Build Pyramid,151,00:46:49,20.73,601
1,Run,Zone 3 Intervals,157,00:13:38,1.73,187
2,Bike,Paris - Easy Effort spin,140,00:31:21,3.95,357
3,Run,Run Test,141,00:22:28,2.71,278
4,Swim,Nat. piscine,136,00:22:11,"1,000",255


In [40]:
workouts_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 796 entries, 2022-03-13 to 2025-03-03
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   WorkoutType              796 non-null    object 
 1   Title                    796 non-null    object 
 2   WorkoutDescription       488 non-null    object 
 3   CoachComments            335 non-null    object 
 4   HeartRateAverage         547 non-null    float64
 5   TimeTotalInHours         572 non-null    float64
 6   DistanceInMeters         572 non-null    float64
 7   PlannedDuration          485 non-null    float64
 8   PlannedDistanceInMeters  144 non-null    float64
dtypes: float64(5), object(4)
memory usage: 62.2+ KB


In [41]:
activities_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, 0 to 581
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   WorkoutType       574 non-null    object
 1   Title             574 non-null    object
 2   HeartRateAverage  549 non-null    object
 3   TimeTotalInHours  574 non-null    object
 4   DistanceInMeters  574 non-null    object
 5   Calories          573 non-null    object
dtypes: object(6)
memory usage: 31.4+ KB


In [42]:
workouts_df['HeartRateAverage'].isna().sum()

np.int64(249)